In [1]:
import os
os.chdir('../')
%pwd

'D:\\projects\\Project-GAN-AnimeFaces'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    kaggle_source: str
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from AnimeFaces.constants import * 
from AnimeFaces.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH , schema_filepath = SCHEMA_FILE_PATH,):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            kaggle_source = config.kaggle_source,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from AnimeFaces import logger
from AnimeFaces.utils.common import get_size
import kaggle

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            DATASET_ORIGINAL_SIZE = 404127
            logger.info(f"Files from URL: {self.config.source_URL} will be downloaded and zipped at {self.config.local_data_file}")
            file = kaggle.api.dataset_download_files(self.config.kaggle_source, path=self.config.unzip_dir)
            logger.info(f"File Has Been Downloaded with size {get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"File already existts of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)            

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-01 21:55:23,955: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-01 21:55:23,957: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-01 21:55:23,959: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-01 21:55:23,960: INFO: common: Directory Created: artifacts]
[2023-10-01 21:55:23,961: INFO: common: Directory Created: artifacts/data_ingestion]
[2023-10-01 21:55:23,962: INFO: 2957387440: File already existts of size: ~ 404127 KB]


KeyboardInterrupt: 